This code creates an ensemble from a selection of models listed: a) customized CNN, b) pretrained VGG16 initialized with ImageNet weights and trained from the scratch, c) Pretrained VGG19 model with ImageNet weights freezed and trained as a classifier, d) Local Binary patterns (LBP)/SVM and e) Histogram of Oriented Gradients (HOG)/SVM classifier. The resulstant ensemble is used to make predictions.

In [ ]:
#load libraries
from keras.models import Model, Input
import os
import statistics
from keras.utils import plot_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D, Dense, Average, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from sklearn.metrics import accuracy_score
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
import numpy as np
import cv2
import glob
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from evaluation import plot_confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.utils import class_weight
from sklearn.svm import NuSVC
from sklearn import preprocessing
from skimage import feature
from sklearn.externals import joblib
%matplotlib inline

In [ ]:
#%% Loading the training data

img_width, img_height = 224, 224
train_data_dir = 'cxr/train'
validation_data_dir = 'cxr/validation'
epochs = 100
batch_size = 16 #in powers of 2 for optimal GPU computation
num_classes= 2

In [ ]:
# Since the models work with the data of the same shape, 
#we define a single input layer that will be used by every model.

input_shape = (224, 224, 3)
model_input = Input(shape=input_shape)

In [ ]:
#%% declaring image data generators, make sure to delcare shuffle=False for validation and testing

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=2,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.5,
      horizontal_flip=False,
      fill_mode='nearest')

# Note that the validation data and test data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',shuffle=False)

#identify the number of samples
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)

#check the class indices
train_generator.class_indices
validation_generator.class_indices

#true labels
Y_test=validation_generator.classes

In [ ]:
#%% assign class weights to balance model training and penalize over-represented classes

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)
print(class_weights)

In [ ]:
#%% define custom model and instantiate it

def custom_cnn(model_input):
    x = BatchNormalization()(model_input)
    x = SeparableConv2D(64, (5, 5), padding='same', activation='relu', name = 'sepconv1')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), name = 'maxpool1')(x)
    x = Dropout(0.25, name = 'conv_dropout1')(x)
    x = BatchNormalization(name = 'custom_batchnorm1')(x)
    x = SeparableConv2D(128, (5, 5), padding='same', activation='relu', name = 'sepconv2')(x)
    x = MaxPooling2D(pool_size=(2, 2), name = 'maxpool2')(x)
    x = Dropout(0.25, name = 'conv_dropout2')(x)
    x = BatchNormalization(name = 'custom_batchnorm2')(x)
    x = SeparableConv2D(256, (5, 5), padding='same', activation='relu', name = 'sepconv3')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), name = 'maxpool3')(x)
    x = Dropout(0.25, name = 'conv_dropout3')(x)
    x = GlobalAveragePooling2D(name = 'custom_GAP')(x)
    x = Dense(256, activation='relu', name = 'custom_dense1')(x)
    x = Dropout(0.5, name = 'custom_dropout1')(x)
    x = Dense(num_classes, activation='softmax', name = 'custom_dense2')(x)
    model = Model(inputs=model_input, outputs=x, name='custom_cnn')
    return model

#instantiate the model
custom_model = custom_cnn(model_input)

#display model summary
custom_model.summary()

#plot the model
plot_model(custom_model, to_file='custom_model.png',show_shapes=True, show_layer_names=False)

In [ ]:
#%% Each model is compiled and trained using the same parameters. 

def compile_and_train(model, num_epochs): 
    
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)  
    model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy']) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{val_acc:.4f}.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                                 save_weights_only=False, save_best_only=True, mode='max', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=batch_size)
    callbacks_list = [checkpoint, tensor_board]
    history = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples // batch_size,
                                  epochs=epochs, validation_data=validation_generator,
                                  class_weight = class_weights,
                                  callbacks=callbacks_list, 
                                  validation_steps=nb_validation_samples // batch_size, verbose=1)                      
    return history

def evaluate_accuracy(model):

    y_pred = model.predict_generator(validation_generator, nb_validation_samples/batch_size, workers=1)
    accuracy = accuracy_score(Y_test,y_pred.argmax(axis=-1))
    return accuracy

def evaluate_mse(model):

    y_pred = model.predict_generator(validation_generator, nb_validation_samples/batch_size, workers=1)
    mse = mean_squared_error(Y_test,y_pred.argmax(axis=-1))
    return mse

def evaluate_msle(model):

    y_pred = model.predict_generator(validation_generator, nb_validation_samples/batch_size, workers=1)
    msle = mean_squared_log_error(Y_test,y_pred.argmax(axis=-1))  
    return msle

In [ ]:
#%% compile and train the custom model
    
_ = compile_and_train(custom_model, num_epochs=epochs)

In [ ]:
#Evaluate the model by loading the best weights and calculating the accuracy and errors on the test set.

custom_model.load_weights('weights/custom_cnn.01-0.5267.h5') #path to the best weights stored

print('The accuracy of the Custom model is: ', evaluate_accuracy(custom_model))
print('The Mean Squared Error of the Custom model is: ', evaluate_mse(custom_model))
print('The Mean Squared Log Error of the Custom model is: ', evaluate_msle(custom_model))

In [ ]:
#%% print classification report and plot confusion matrix

custom_y_pred = custom_model.predict_generator(validation_generator, nb_validation_samples/batch_size, workers=1)
target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks
print(classification_report(Y_test,custom_y_pred.argmax(axis=-1),target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,custom_y_pred.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for custom model without normalization')

plt.show()

In [ ]:
#%% compute the ROC-AUC values

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test, custom_y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
fig=plt.figure(figsize=(15,10), dpi=100)
ax = fig.add_subplot(1, 1, 1)
# Major ticks every 0.05, minor ticks every 0.05
major_ticks = np.arange(0.0, 1.0, 0.05)
minor_ticks = np.arange(0.0, 1.0, 0.05)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(fpr[1], tpr[1], color='red',
         lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics for Custom model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#%% define the second model : VGG16 

def vgg16_cnn(model_input):
    vgg16_cnn = VGG16(weights='imagenet', include_top=False, input_tensor=model_input)
    vgg16_cnn = Model(inputs=vgg16_cnn.input, outputs=vgg16_cnn.get_layer('block5_conv3').output)
    x = vgg16_cnn.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=vgg16_cnn.input, outputs=predictions, name='vgg16_custom')
    return model

#instantiate the model
vgg16_custom_model = vgg16_cnn(model_input)

#display model summary
vgg16_custom_model.summary()

#plot model
plot_model(vgg16_custom_model, to_file='vgg16_custom_model.png',show_shapes=True, show_layer_names=False)

In [ ]:
#compile and train the model
_ = compile_and_train(vgg16_custom_model, num_epochs=epochs)

In [ ]:
#evaluate the accuracy and errors on the test set

vgg16_custom_model.load_weights('weights/vgg16_custom.01-0.9817.h5')

#renaming the model layers to prevent issues while doing model ensemble
vgg16_custom_model.get_layer(name='block1_conv1').name='block1_conv1VGG'  
vgg16_custom_model.get_layer(name='block1_conv2').name='block1_conv2VGG' 
vgg16_custom_model.get_layer(name='block2_conv1').name='block2_conv1VGG' 
vgg16_custom_model.get_layer(name='block2_conv2').name='block2_conv2VGG' 
vgg16_custom_model.get_layer(name='block3_conv1').name='block3_conv1VGG' 
vgg16_custom_model.get_layer(name='block3_conv2').name='block3_conv2VGG' 
vgg16_custom_model.get_layer(name='block3_conv3').name='block3_conv3VGG' 
vgg16_custom_model.get_layer(name='block4_conv1').name='block4_conv1VGG' 
vgg16_custom_model.get_layer(name='block4_conv2').name='block4_conv2VGG' 
vgg16_custom_model.get_layer(name='block4_conv3').name='block4_conv3VGG' 
vgg16_custom_model.get_layer(name='block5_conv1').name='block5_conv1VGG' 
vgg16_custom_model.get_layer(name='block5_conv2').name='block5_conv2VGG' 
vgg16_custom_model.get_layer(name='block5_conv3').name='block5_conv3VGG' 
vgg16_custom_model.get_layer(name='block1_pool').name='block1_poolVGG' 
vgg16_custom_model.get_layer(name='block2_pool').name='block2_poolVGG' 
vgg16_custom_model.get_layer(name='block3_pool').name='block3_poolVGG' 
vgg16_custom_model.get_layer(name='block4_pool').name='block4_poolVGG' 

#display model summary
vgg16_custom_model.summary()

print('The accuracy of the VGG16 model is: ', evaluate_accuracy(vgg16_custom_model))
print('The Mean Squared Error of the VGG16 model is: ', evaluate_mse(vgg16_custom_model))
print('The Mean Squared Log Error of the VGG16 model is: ', evaluate_msle(vgg16_custom_model))

In [ ]:
#%% print classification report and plot confusion matrix

vgg16_y_pred = vgg16_custom_model.predict_generator(validation_generator, nb_validation_samples/batch_size, workers=1)
target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks
print(classification_report(Y_test,vgg16_y_pred.argmax(axis=-1),target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,vgg16_y_pred.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for vgg16 without normalization')

plt.show()

In [ ]:
#%% compute the ROC-AUC values

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test, vgg16_y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
fig=plt.figure(figsize=(15,10), dpi=100)
ax = fig.add_subplot(1, 1, 1)
# Major ticks every 0.05, minor ticks every 0.05
major_ticks = np.arange(0.0, 1.0, 0.05)
minor_ticks = np.arange(0.0, 1.0, 0.05)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(fpr[1], tpr[1], color='red',
         lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics for VGG16 model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#%% The third model is the xception model to be trained as a classifier

def vgg19_cnn(model_input):
    vgg19_cnn = VGG19(weights='imagenet', include_top=False, input_tensor=model_input)
    x = vgg19_cnn.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.25)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=vgg19_cnn.input, outputs=predictions, name='vgg19_custom')
    #freeze the training layers and train only the classifier
    for layer in vgg19_cnn.layers:
        layer.trainable = False   
    return model

#instantiate the model
vgg19_custom_model = vgg19_cnn(model_input)

#plot model summary
vgg19_custom_model.summary()
#plot_model(vgg19_custom_model, to_file='vgg19_custom_model.png',show_shapes=True, show_layer_names=False)

In [ ]:
#compile and train the model
_ = compile_and_train(vgg19_custom_model, num_epochs=epochs)

In [ ]:
#evaluate the accuracy and errors on the test set
vgg19_custom_model.load_weights('weights/vgg19_custom.01-0.7652.h5')

#renaming the model layers to prevent issues while doing model ensemble
vgg19_custom_model.get_layer(name='block1_conv1').name='block1_conv1VGG19'  
vgg19_custom_model.get_layer(name='block1_conv2').name='block1_conv2VGG19' 
vgg19_custom_model.get_layer(name='block2_conv1').name='block2_conv1VGG19' 
vgg19_custom_model.get_layer(name='block2_conv2').name='block2_conv2VGG19' 
vgg19_custom_model.get_layer(name='block3_conv1').name='block3_conv1VGG19' 
vgg19_custom_model.get_layer(name='block3_conv2').name='block3_conv2VGG19' 
vgg19_custom_model.get_layer(name='block3_conv3').name='block3_conv3VGG19' 
vgg19_custom_model.get_layer(name='block3_conv4').name='block3_conv4VGG19' 
vgg19_custom_model.get_layer(name='block4_conv1').name='block4_conv1VGG19' 
vgg19_custom_model.get_layer(name='block4_conv2').name='block4_conv2VGG19' 
vgg19_custom_model.get_layer(name='block4_conv3').name='block4_conv3VGG19' 
vgg19_custom_model.get_layer(name='block4_conv4').name='block4_conv4VGG19' 
vgg19_custom_model.get_layer(name='block5_conv1').name='block5_conv1VGG19' 
vgg19_custom_model.get_layer(name='block5_conv2').name='block5_conv2VGG19' 
vgg19_custom_model.get_layer(name='block5_conv3').name='block5_conv3VGG19' 
vgg19_custom_model.get_layer(name='block5_conv4').name='block5_conv4VGG19'
vgg19_custom_model.get_layer(name='block1_pool').name='block1_poolVGG19' 
vgg19_custom_model.get_layer(name='block2_pool').name='block2_poolVGG19' 
vgg19_custom_model.get_layer(name='block3_pool').name='block3_poolVGG19' 
vgg19_custom_model.get_layer(name='block4_pool').name='block4_poolVGG19' 

print('The accuracy of the vgg19 model is: ', evaluate_accuracy(vgg19_custom_model))
print('The Mean Squared Error of the vgg19 model is: ', evaluate_mse(vgg19_custom_model))
print('The Mean Squared Log Error of the vgg19 model is: ', evaluate_msle(vgg19_custom_model))

In [ ]:
#%% print classification report and plot confusion matrix

vgg19_y_pred = vgg19_custom_model.predict_generator(validation_generator, nb_validation_samples/batch_size, workers=1)
print(vgg19_y_pred.shape)

target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks
print(classification_report(Y_test,vgg19_y_pred.argmax(axis=-1),target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,vgg19_y_pred.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for VGG19 without normalization')

plt.show()

In [ ]:
#%% compute the ROC-AUC values

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test, vgg19_y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
fig=plt.figure(figsize=(15,10), dpi=100)
ax = fig.add_subplot(1, 1, 1)
# Major ticks every 0.05, minor ticks every 0.05
major_ticks = np.arange(0.0, 1.0, 0.05)
minor_ticks = np.arange(0.0, 1.0, 0.05)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(fpr[1], tpr[1], color='red',
         lw=lw, label='ROC curve (area = %0.4f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics for VGG19 model')
plt.legend(loc="lower right")
plt.show()

Local binary patterns (LBP) is a type of visual descriptor used for classification in computer vision. It has been found to be a powerful feature for texture classification; it has further been determined that when LBP is combined with the Histogram of oriented gradients (HOG) descriptor, it improves the detection performance considerably on some datasets.The LBP feature vector is created in the following manner: 

Divide the examined image into cells (e.g. 16x16 pixels for each cell).

For each pixel in a cell, compare the pixel to each of its 8 neighbors (on its left-top, left-middle, left-bottom, right-top, etc.). 

Follow the pixels along a circle, i.e. clockwise or counter-clockwise.

Where the center pixel's value is greater than the neighbor's value, write "0". Otherwise, write "1". This gives an 8-digit binary number (which is usually converted to decimal for convenience).

Compute the histogram, over the cell, of the frequency of each "number" occurring (i.e., each combination of which pixels are smaller and which are greater than the center). This histogram can be seen as a 256-dimensional feature vector.
Optionally normalize the histogram.

Concatenate (normalized) histograms of all cells. This gives a feature vector for the entire window.

The feature vector can now be processed using the Support vector machine, extreme learning machines, or some other machine-learning algorithm to classify images. Such classifiers can be used for face recognition or texture analysis. 

In [ ]:
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
 
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
 
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
 
        # return the histogram of Local Binary Patterns
        return hist

In [ ]:
#%% empty list to hold feature vectors and train labels
        
lbp_train_features = []
lbp_train_labels = []

# initialize our LocalBinary Pattern descriptor using numPoints and radius.
# radius = 3 and numpoints = 8 * radius (=24) works well for this problem

desc = LocalBinaryPatterns(24, 3) 

# loop over the training dataset
print ("[STATUS] Started extracting LBP features....")
data_dir = os.path.join(os.getcwd(),train_data_dir)

cur_label = os.listdir(data_dir)
cur_label.sort()

for label in cur_label:
    dirpath = os.path.join(data_dir,label)
    i = 1
    for file in glob.glob(dirpath + "/*.png"):
            print ("Processing Image - {} in {}".format(i, label))
            
            # read the training image
            image = cv2.imread(file)
            
            # convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # extract Local Binary Patterns from the image
            lbp_features = desc.describe(gray)

            # append the feature vector and label
            lbp_train_features.append(lbp_features)
            lbp_train_labels.append(label)

            # show loop update
            i += 1

# have a look at the size of our feature vector and labels
print ("Training features: {}".format(np.array(lbp_train_features).shape))
print ("Training labels: {}".format(np.array(lbp_train_labels).shape))

In [ ]:
#%% create the classifier
print ("[STATUS] Creating the classifier..")
clf_nusvm_lbp = NuSVC(nu=0.4, kernel='linear', probability=True, class_weight='balanced', random_state=9)

The value of nu = 04 and linear kernel gave the best results here. The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)). The probability is set to true to compute class probabilities however the function is not available for all classifiers incuding kNN but only SVC and NuSVC has it according to
https://stackoverflow.com/questions/15015710/how-can-i-know-probability-of-class-predicted-by-predict-function-in-support-v

The trained model can also be stored using pickle like this:

lbp_svm_filename = 'LBP_SVM_MODEL.sav'

pickle.dump(clf_nusvm_lbp, open(lbp_svm_filename, 'wb'))

The saved model can be loaded from disk like this:

lbp_svm_model = pickle.load(open(lbp_svm_filename, 'rb'))


In [ ]:
print ("[STATUS] Fitting data/label to model..")

#create an array of training features and labels
lbp_train_features = np.array(lbp_train_features)
lbp_train_labels = np.array(lbp_train_labels)

#randomize the training data
lbp_idxs = np.random.permutation(len(lbp_train_features))
lbp_train_features = lbp_train_features[lbp_idxs]
lbp_train_labels = lbp_train_labels[lbp_idxs]

#fit the classifier
clf_nusvm_lbp.fit(lbp_train_features, lbp_train_labels)

#save the model to disk
lbp_svm_filename = 'LBP_SVM_MODEL.sav'
joblib.dump(clf_nusvm_lbp, lbp_svm_filename)

In [ ]:
#%% load the saved model and predict on the test data 
lbp_svm_model = joblib.load(lbp_svm_filename)

#load test data
data_dir = os.path.join(os.getcwd(),validation_data_dir)

lbp_y_pred = []
lbp_y_pred1 = []
Y_test_lbp = []
lbp_class_labels = []

cur_label = os.listdir(data_dir)
cur_label.sort()

for label in cur_label:
    dirpath = os.path.join(data_dir,label)
    i = 1
    lbp_class_labels.append(label)
    for file in glob.glob(dirpath + "/*.png"): #modify for png
            print ("Processing Image - {} in {}".format(i, label))
            
            # read the training image
            image = cv2.imread(file)
            Y_test_lbp.append(label)
            
            # convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # extract LBP features from the image
            lbp_test_features = desc.describe(gray)

            # evaluate the model and predict label
            lbp_prediction = lbp_svm_model.predict(lbp_test_features.reshape(1, -1))[0]
            lbp_prediction1 = lbp_svm_model.predict_proba(lbp_test_features.reshape(1, -1))[0]
            lbp_y_pred.append(lbp_prediction)
            lbp_y_pred1.append(lbp_prediction1)
            # show loop update
            i += 1
            
le = preprocessing.LabelEncoder()
le.fit(lbp_class_labels)

localbinary_y_pred = le.transform(lbp_y_pred)
Y_test_lbp = le.transform(Y_test_lbp)
localbinary_y_pred1 = np.array(lbp_y_pred1)

#print shape to check
print(localbinary_y_pred.shape)
print(localbinary_y_pred1.shape)
print(Y_test_lbp.shape)

#save the predictions
np.savetxt('localbinary_y_pred.csv',localbinary_y_pred1.argmax(axis=-1),fmt='%i',delimiter = ",")
np.savetxt('Y_test_lbp.csv',Y_test_lbp,fmt='%i',delimiter = ",")

In [ ]:
#%% measure accuracy

LBP_SVM_model_accuracy=accuracy_score(Y_test_lbp,localbinary_y_pred)
print("The accuracy of LBP SVM model is = ", LBP_SVM_model_accuracy)

#plot confusion matrix
target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks

#print classification report
print(classification_report(Y_test_lbp,localbinary_y_pred1.argmax(axis=-1),target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test_lbp,localbinary_y_pred1.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=300)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for LBP SVM model without normalization')

plt.show()

In [ ]:
#%% compute the ROC-AUC values

lbp_fpr = dict()
lbp_tpr = dict()
lbp_roc_auc = dict()
for i in range(num_classes):
    lbp_fpr[i], lbp_tpr[i], _ = roc_curve(Y_test_lbp, localbinary_y_pred1[:,i])
    lbp_roc_auc[i] = auc(lbp_fpr[i], lbp_tpr[i])
fig=plt.figure(figsize=(15,10), dpi=300)
ax = fig.add_subplot(1, 1, 1)
# Major ticks every 0.05, minor ticks every 0.05
major_ticks = np.arange(0.0, 1.0, 0.05)
minor_ticks = np.arange(0.0, 1.0, 0.05)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(lbp_fpr[1], lbp_tpr[1], color='cyan',
         lw=lw, label='ROC curve (area = %0.4f)' % lbp_roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics for LBP SVM model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#%% evaluate error

LBP_SVM_model_mean_squared_error = mean_squared_error(Y_test_lbp,localbinary_y_pred)  
LBP_SVM_model_mean_squared_log_error = mean_squared_log_error(Y_test_lbp,localbinary_y_pred)  
print("The mean squared error of LBP_NUSVMV model is = ", LBP_SVM_model_mean_squared_error)
print("The mean squared log error of LBP_NUSVMV model is = ",LBP_SVM_model_mean_squared_log_error)

Histogram of oriented gradients (HOG) is a feature descriptor used in computer vision and image processing for the purpose of object detection. The technique counts occurrences of gradient orientation in localized portions of an image. This method is similar to that of edge orientation histograms, scale-invariant feature transform descriptors, and shape contexts, but differs in that it is computed on a dense grid of uniformly spaced cells and uses overlapping local contrast normalization for improved accuracy. 

The essential thought behind the histogram of oriented gradients descriptor is that local object appearance and shape within an image can be described by the distribution of intensity gradients or edge directions. The image is divided into small connected regions called cells, and for the pixels within each cell, a histogram of gradient directions is compiled. The descriptor is the concatenation of these histograms. 

For improved accuracy, the local histograms can be contrast-normalized by calculating a measure of the intensity across a larger region of the image, called a block, and then using this value to normalize all cells within the block. This normalization results in better invariance to changes in illumination and shadowing. 

The HOG descriptor has a few key advantages over other descriptors. Since it operates on local cells, it is invariant to geometric and photometric transformations, except for object orientation. Such changes would only appear in larger spatial regions.

In [ ]:
#%% HOG feature extraction

hog_train_features = []
hog_train_labels = []

def extract_features_hog(image):
    hog_orientation = feature.hog(image, orientations=9, pixels_per_cell=(8,8),
                                  cells_per_block=(3,3), visualise=None, transform_sqrt=False, feature_vector=True)
    
    # take the mean of it and return it
    return hog_orientation

In [ ]:
#%% loop over the training dataset

print ("[STATUS] Started extracting HOG textures..")
data_dir = os.path.join(os.getcwd(),train_data_dir)

cur_label = os.listdir(data_dir)
cur_label.sort()

for label in cur_label:
    dirpath = os.path.join(data_dir,label)
    i = 1
    for file in glob.glob(dirpath + "/*.png"): #change for png files
            print ("Processing Image - {} in {}".format(i, label))
            
            # read the training image
            image = cv2.imread(file)
            
            # convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # extract HOG features from the image
            hog_features = extract_features_hog(gray)

            # append the feature vector and label
            hog_train_features.append(hog_features)
            hog_train_labels.append(label)

            # show loop update
            i += 1
                
# have a look at the size of our feature vector and labels
print ("Training features: {}".format(np.array(hog_train_features).shape))
print ("Training labels: {}".format(np.array(hog_train_labels).shape))

In [ ]:
#%% create the classifier
print ("[STATUS] Creating the classifier..")
clf_nusvm_hog = NuSVC(nu=0.4, kernel='linear', probability=True, class_weight='balanced', random_state=9)

print ("[STATUS] Fitting data/label to model..")
#create an array of training features and labels
hog_train_features = np.array(hog_train_features)
hog_train_labels = np.array(hog_train_labels)

#randomize the training data to make it robust
hog_idxs = np.random.permutation(len(hog_train_features))
hog_train_features = hog_train_features[hog_idxs]
hog_train_labels = hog_train_labels[hog_idxs]

#fit the classifier
clf_nusvm_hog.fit(hog_train_features, hog_train_labels)

#save the model to disk
hog_svm_filename = 'HOG_SVM_MODEL.sav'
joblib.dump(clf_nusvm_hog, hog_svm_filename)

In [ ]:
#%% load the saved model and predict on the test data 
hog_svm_model = joblib.load(hog_svm_filename)

# predict on the test data
data_dir = os.path.join(os.getcwd(),validation_data_dir)

hog_y_pred = []
hog_y_pred1 = []
Y_test_hog = [] #ground truth labels
hog_class_labels = []

cur_label = os.listdir(data_dir)
cur_label.sort()

for label in cur_label:
    dirpath = os.path.join(data_dir,label)
    i = 1
    hog_class_labels.append(label)
    for file in glob.glob(dirpath + "/*.png"):
            print ("Processing Image - {} in {}".format(i, label))
            
            # read the training image
            image = cv2.imread(file)
            Y_test_hog.append(label)
            
            # convert the image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # extract HOG features from the image
            hog_test_features = extract_features_hog(gray)
            
            # make predictions
            hog_prediction = hog_svm_model.predict(hog_test_features.reshape(1, -1))[0]
            hog_prediction1 = hog_svm_model.predict_proba(hog_test_features.reshape(1, -1))[0]
            hog_y_pred.append(hog_prediction)
            hog_y_pred1.append(hog_prediction1)
            
            # show loop update
            i += 1

le = preprocessing.LabelEncoder()
le.fit(hog_class_labels)

hog_y_pred = le.transform(hog_y_pred)
Y_test_hog = le.transform(Y_test_hog)
hog_y_pred1 = np.array(hog_y_pred1)

#print shape to check
print(hog_y_pred.shape)
print(hog_y_pred1.shape)
print(Y_test_hog.shape)

#save the predictions
np.savetxt('hog_y_pred.csv',hog_y_pred1.argmax(axis=-1),fmt='%i',delimiter = ",")
np.savetxt('Y_test_hog.csv',Y_test_hog,fmt='%i',delimiter = ",")

In [ ]:
#%% measure accuracy

hog_svm_model_accuracy=accuracy_score(Y_test_hog,hog_y_pred)
print("The accuracy of HOG SVM is = ", hog_svm_model_accuracy)

#plot confusion matrix
target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks

#print classification report
print(classification_report(Y_test_hog,hog_y_pred,target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test_hog,hog_y_pred)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=300)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for HOG SVM model without normalization')

plt.show()

In [ ]:
#%% compute roc_auc values

hog_fpr = dict()
hog_tpr = dict()
hog_roc_auc = dict()
for i in range(num_classes):
    hog_fpr[i], hog_tpr[i], _ = roc_curve(Y_test_hog, hog_y_pred1[:,i])
    hog_roc_auc[i] = auc(hog_fpr[i], hog_tpr[i])
fig=plt.figure(figsize=(15,10), dpi=300)
ax = fig.add_subplot(1, 1, 1)
major_ticks = np.arange(0.0, 1.0, 0.1)
minor_ticks = np.arange(0.0, 1.0, 0.1)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(hog_fpr[1], hog_tpr[1], color='green',
         lw=lw, label='ROC curve (area = %0.4f)' % hog_roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics for HOG SVM model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#%% evaluate error and print

HOG_SVM_model_mean_squared_error = mean_squared_error(Y_test_hog,hog_y_pred)  
HOG_SVM_model_mean_squared_log_error = mean_squared_log_error(Y_test_hog,hog_y_pred)  
print("The mean squared error of HOG_NUSVM model is = ",HOG_SVM_model_mean_squared_error)
print("The mean squared log error of HOG_NUSVM model is = ",HOG_SVM_model_mean_squared_log_error)

In [ ]:
#dummy assignment of variable for majority voting

y_pred1 = custom_y_pred
vgg16_y_pred1 = vgg16_y_pred
vgg19_y_pred1 = vgg19_y_pred

print("The shape of custom model prediction y_pred is  = ", y_pred1.shape)
print("The shape of VGG16 model prediction vgg16_y_pred is  = ", vgg16_y_pred1.shape)
print("The shape of VGG19 model prediction vgg19_y_pred is = ", vgg19_y_pred1.shape)
print("The shape of LBP SVM model prediction localbinary_y_pred is  = ", localbinary_y_pred.shape)
print("The shape of HOG SVM model prediction hog_y_pred is  = ", hog_y_pred.shape)

In [ ]:
y_pred1 = y_pred1.argmax(axis=-1)
print(y_pred1)
vgg16_y_pred1 = vgg16_y_pred1.argmax(axis=-1)
print(vgg16_y_pred1)
vgg19_y_pred1 = vgg19_y_pred1.argmax(axis=-1)
print(vgg19_y_pred1)

#max voting begins
max_voting_pred = np.array([])
for i in range(0,len(validation_generator.filenames)):
    max_voting_pred = np.append(max_voting_pred, 
                                statistics.mode([y_pred1[i], vgg16_y_pred1[i], vgg19_y_pred1[i],
                                                 localbinary_y_pred[i], hog_y_pred[i]]))
    
ensemble_model_max_voting_accuracy = accuracy_score(Y_test,max_voting_pred)
print("The max voting accuracy of the ensemble model is  = ", ensemble_model_max_voting_accuracy)

#plot confusion matrix
target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks

#print classification report
print(classification_report(Y_test,max_voting_pred,target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,max_voting_pred)
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for Max Voting ensemble without normalization')

plt.show()

#save the predictions
np.savetxt('max_voting_y_pred.csv',max_voting_pred,fmt='%i',delimiter = ",")
np.savetxt('Y_test_maxvoting.csv',Y_test,fmt='%i',delimiter = ",")

In [ ]:
#%% evaluate error

ensemble_model_maxvoting_mean_squared_error = mean_squared_error(Y_test,max_voting_pred)  
ensemble_model_maxvoting_mean_squared_log_error = mean_squared_log_error(Y_test,max_voting_pred)  
print("The max voting mean squared error of the ensemble model is  = ", ensemble_model_maxvoting_mean_squared_error)
print("The max voting mean squared log error of the ensemble model is  = ", ensemble_model_maxvoting_mean_squared_log_error)

Performing Simple Averaging

In [ ]:
#%% simple averaging

average_pred=(custom_y_pred + vgg16_y_pred + vgg19_y_pred + localbinary_y_pred1 + hog_y_pred1)/5
ensemble_model_averaging_accuracy = accuracy_score(Y_test,average_pred.argmax(axis=-1))
print("The averaging accuracy of the ensemble model is  = ", ensemble_model_averaging_accuracy)

#plot confusion matrix
target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks

#print classification report
print(classification_report(Y_test,average_pred.argmax(axis=-1),target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,average_pred.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for Average Ensemble without normalization')

plt.show()
#save the predictions
np.savetxt('averagring_y_pred.csv',average_pred.argmax(axis=-1),fmt='%i',delimiter = ",")
np.savetxt('Y_test_averaging.csv',Y_test,fmt='%i',delimiter = ",")

In [ ]:
#%% plot roc auc

avg_fpr = dict()
avg_tpr = dict()
avg_roc_auc = dict()
for i in range(num_classes):
    avg_fpr[i], avg_tpr[i], _ = roc_curve(Y_test, average_pred[:, i])
    avg_roc_auc[i] = auc(avg_fpr[i], avg_tpr[i])
fig=plt.figure(figsize=(15,10), dpi=300)
ax = fig.add_subplot(1, 1, 1)
major_ticks = np.arange(0.0, 1.0, 0.1)
minor_ticks = np.arange(0.0, 1.0, 0.1)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(avg_fpr[1], avg_tpr[1], color='cyan',
         lw=lw, label='ROC curve (area = %0.4f)' % avg_roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics for VGG19 model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#%% evaluate error
ensemble_model_averaging_mean_squared_error = mean_squared_error(Y_test,average_pred.argmax(axis=-1))  
ensemble_model_averaging_mean_squared_log_error = mean_squared_log_error(Y_test,average_pred.argmax(axis=-1))  
print("The averaging mean squared error of the ensemble model is  = ", ensemble_model_averaging_mean_squared_error)
print("The averaging mean squared log error of the ensemble model is  = ", ensemble_model_averaging_mean_squared_log_error)

Weighted Averaging: This is an extension of the averaging method. All models are assigned different weights defining the importance of each model for prediction. With VGG16 having generic knowledge about images, its predictions is given more importance as compared to the other models.Next, VGG19 is given higher weights followed by the other models. 


In [ ]:
weighted_average_pred=(custom_y_pred * 0.05 + vgg16_y_pred * 0.4 + vgg19_y_pred * 0.3
                       + localbinary_y_pred1 * 0.2 + hog_y_pred1 * 0.05)
ensemble_model_weighted_averaging_accuracy = accuracy_score(Y_test,weighted_average_pred.argmax(axis=-1))
print("The weighted averaging accuracy of the ensemble model is  = ", ensemble_model_weighted_averaging_accuracy)

#plot confusion matrix
target_names = ['class 0(abnormal)', 'class 1(normal)'] #modify according to tasks

#print classification report
print(classification_report(Y_test,weighted_average_pred.argmax(axis=-1),target_names=target_names, digits=4))

# Compute confusion matrix
cnf_matrix = confusion_matrix(Y_test,weighted_average_pred.argmax(axis=-1))
np.set_printoptions(precision=4)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10), dpi=100)
plot_confusion_matrix(cnf_matrix, classes=target_names,
                      title='Confusion matrix for Weighted Average Ensemble without normalization')

plt.show()

In [ ]:
#%% save the predictions

np.savetxt('weighted_averaging_y_pred.csv',weighted_average_pred.argmax(axis=-1),fmt='%i',delimiter = ",")
np.savetxt('Y_test_weighted_averaging.csv',Y_test,fmt='%i',delimiter = ",")

In [ ]:
#%% plot roc curves

wavg_fpr = dict()
wavg_tpr = dict()
wavg_roc_auc = dict()
for i in range(num_classes):
    wavg_fpr[i], wavg_tpr[i], _ = roc_curve(Y_test, weighted_average_pred[:, i])
    wavg_roc_auc[i] = auc(wavg_fpr[i], wavg_tpr[i])
fig=plt.figure(figsize=(15,10), dpi=100)
ax = fig.add_subplot(1, 1, 1)
major_ticks = np.arange(0.0, 1.0, 0.1)
minor_ticks = np.arange(0.0, 1.0, 0.1)
ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='both')
lw = 1 
plt.plot(wavg_fpr[1], wavg_tpr[1], color='magenta',
         lw=lw, label='ROC curve (area = %0.4f)' % wavg_roc_auc[1])
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristics for VGG19 model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#%% evaluate error
ensemble_model_weighted_average_mean_squared_error = mean_squared_error(Y_test,weighted_average_pred.argmax(axis=-1))  
ensemble_model_weighted_average_mean_squared_log_error = mean_squared_log_error(Y_test,weighted_average_pred.argmax(axis=-1))  
print("The weighted averaging mean squared error of the ensemble model is  = ", ensemble_model_weighted_average_mean_squared_error)
print("The weighted averaging mean squared log error of the ensemble model is  = ", ensemble_model_weighted_average_mean_squared_log_error)


Every model has its own weaknesses. The reasoning behind using an ensemble is that by stacking different models representing different hypotheses about the data, we can find a better hypothesis that is not in the hypothesis space of the models from which the ensemble is built. By using a very basic ensemble, a much lower error rate was achieved than when a single model was used. This proves effectiveness of ensembling. Of course, there are some practical considerations to keep in mind when using an ensemble for your machine learning task. Since ensembling means stacking multiple models together, it also means that the input data needs to be forward-propagated for each model. This increases the amount of compute that needs to be performed and, consequently, evaluation (predicition) time. Increased evaluation time is not critical if you use an ensemble in research or in a Kaggle competition. However, it is a very critical factor when designing a commercial product. Another consideration is increased size of the final model which, again, might be a limiting factor for ensemble use in a commercial product.